In [3]:
import tkinter as tk                                  # mengimpor modul GUI
from tkinter import ttk, messagebox, filedialog       # komponen GUI modern & popup
from matplotlib.figure import Figure                  # membuat figure matplotlib
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg  # canvas untuk tkinter
from mpl_toolkits.mplot3d import Axes3D               # modul plot 3D
import numpy as np                                    # mengimpor numpy untuk perhitungan matematis
import pandas as pd                                   # mengimpor pandas untuk membaca file CSV


# DATA CLASS UNTUK MENYIMPAN ORIENTASI

class DataOrientasi:
    def __init__(self):
        self.strikes = []                              # list untuk strike
        self.dips = []                                 # list untuk dip

    def add(self, strike, dip):
        if not (0 <= strike <= 360):                   # validasi strike
            raise ValueError("Strike harus 0–360°")
        if not (0 <= dip <= 90):                       # validasi dip
            raise ValueError("Dip harus 0–90°")

        self.strikes.append(float(strike))             # menyimpan strike
        self.dips.append(float(dip))                  # menyimpan dip

    def clear(self):
        self.strikes = []                              # menghapus semua strike
        self.dips = []                                 # menghapus semua dip

    def load_csv(self, path):
        df = pd.read_csv(path)                         # membaca CSV
        if "strike" not in df or "dip" not in df:      # validasi kolom
            raise ValueError("CSV harus memiliki kolom 'strike' dan 'dip'")

        self.clear()                                   # mengosongkan data lama
        for s, d in zip(df["strike"], df["dip"]):      # loop setiap baris
            self.add(float(s), float(d))               # menambahkan data


# CLASS UNTUK PLOT 3D STEREONET

class StereonetPlotter3D:
    def __init__(self, fig=None):
        self.fig = fig if fig is not None else Figure(figsize=(6, 6), dpi=100)  # figure utama
        self.ax = self.fig.add_subplot(111, projection='3d')                    # axes 3D

    def clear(self):
        self.fig.clf()                                   # membersihkan figure
        self.ax = self.fig.add_subplot(111, projection='3d')  # rebuild axes 3D

    # -------------------------
    def plot_hemisphere(self):
        u = np.linspace(0, 2*np.pi, 60)                   # sudut horizontal
        v = np.linspace(0, np.pi/2, 60)                   # sudut vertikal (0–90°)

        x = np.outer(np.cos(u), np.sin(v))                # koordinat X
        y = np.outer(np.sin(u), np.sin(v))                # koordinat Y
        z = -np.outer(np.ones(np.size(u)), np.cos(v))     # Z dibalik → lower hemisphere

        self.ax.plot_surface(x, y, z, color="lightgray", alpha=0.3)  # gambar hemisphere

    # -------------------------
    def pole(self, strike, dip):
        strike = np.radians(strike)                       # konversi ke radian
        dip = np.radians(dip)

        x = -np.sin(dip) * np.sin(strike)                 # rumus pole
        y = -np.sin(dip) * np.cos(strike)
        z = np.cos(dip)

        self.ax.scatter([x], [y], [z], color='red', s=40) # plot titik pole

    # -------------------------
    def great_circle(self, strike, dip):
        strike = np.radians(strike)                       # radian
        dip = np.radians(dip)

        # hitung pole
        pole = np.array([
            -np.sin(dip) * np.sin(strike),
            -np.sin(dip) * np.cos(strike),
            np.cos(dip)
        ])

        phi = np.linspace(0, 2*np.pi, 200)                # parameter lingkaran besar
        gc = []                                           # list untuk titik GC

        north = np.array([0, 0, 1])                       # arah utara
        v1 = np.cross(pole, north)                        # basis 1 bidang
        v1 /= np.linalg.norm(v1)                          # normalisasi
        v2 = np.cross(pole, v1)                           # basis 2 bidang

        for p in phi:                                     # loop 200 titik
            pt = v1 * np.cos(p) + v2 * np.sin(p)          # formula lingkaran besar
            if pt[2] > 0:                                 # jika di atas hemisphere
                pt *= -1                                  # balik ke bawah
            gc.append(pt)

        gc = np.array(gc)
        self.ax.plot(gc[:, 0], gc[:, 1], gc[:, 2], color='blue')  # gambar great circle

    # -------------------------
    def plot_planes_and_poles(self, strikes, dips):
        self.clear()                                      # membersihkan plot lama
        self.plot_hemisphere()                            # gambar hemisphere

        for s, d in zip(strikes, dips):                    # loop semua data
            self.great_circle(s, d)                       # gambar bidang
            self.pole(s, d)                               # gambar pole

        self.ax.set_box_aspect([1, 1, 1])                 # proporsi 3D seimbang
        self.ax.set_axis_off()                            # menghilangkan axis
        self.ax.set_title("3D Stereonet — Plane & Pole")  # memberi judul plot


# GUI APPLICATION (TKINTER)

class StereonetApp:
    def __init__(self, root):
        self.root = root
        root.title("Stereonet 3D GUI — Strike & Dip Plotter")   # memberi judul window

        self.data = DataOrientasi()                             # data orientasi
        self.figure = Figure(figsize=(6, 6), dpi=100)           # figure plot
        self.plotter = StereonetPlotter3D(self.figure)          # plotter 3D

        # ------------------ Layout GUI ------------------
        self.left = ttk.Frame(root, padding=10)                 # panel kiri
        self.left.grid(row=0, column=0, sticky="ns")

        self.right = ttk.Frame(root)                            # panel kanan (plot)
        self.right.grid(row=0, column=1, sticky="nsew")
        root.columnconfigure(1, weight=1)
        root.rowconfigure(0, weight=1)

        # ------------------ Input strike/dip ------------------
        ttk.Label(self.left, text="Strike (°)").grid(row=0, column=0)
        self.strike_var = tk.StringVar()
        ttk.Entry(self.left, textvariable=self.strike_var, width=12).grid(row=1, column=0)

        ttk.Label(self.left, text="Dip (°)").grid(row=2, column=0)
        self.dip_var = tk.StringVar()
        ttk.Entry(self.left, textvariable=self.dip_var, width=12).grid(row=3, column=0)

        ttk.Button(self.left, text="Add (single)", command=self.add_single).grid(row=4, column=0, pady=5)
        ttk.Button(self.left, text="Plot current", command=self.plot_current).grid(row=5, column=0, pady=5)
        ttk.Button(self.left, text="Clear data", command=self.clear_data).grid(row=6, column=0, pady=5)

        ttk.Separator(self.left, orient="horizontal").grid(row=7, column=0, pady=10, sticky="ew")

        ttk.Button(self.left, text="Load CSV", command=self.load_csv).grid(row=8, column=0, pady=5)
        ttk.Button(self.left, text="Save plot image", command=self.save_plot).grid(row=9, column=0, pady=5)

        ttk.Label(self.left, text="List (strike, dip):").grid(row=10, column=0, pady=(10, 0))
        self.listbox = tk.Listbox(self.left, width=18, height=15)  # listbox data
        self.listbox.grid(row=11, column=0)

        # canvas untuk plot Matplotlib
        self.canvas = FigureCanvasTkAgg(self.figure, master=self.right)
        self.canvas.get_tk_widget().pack(fill="both", expand=True)

    # BUTTON FUNCTIONS
    
    def add_single(self):
        try:
            s = float(self.strike_var.get())                   # mengambil strike
            d = float(self.dip_var.get())                      # mengambil dip
            self.data.add(s, d)                                # menyimpan data
            self.listbox.insert(tk.END, f"{s:.1f}, {d:.1f}")   # menampilkan di listbox
            self.strike_var.set("")                            # reset input
            self.dip_var.set("")
        except Exception as e:
            messagebox.showerror("Error", str(e))              # menampilkan error

    # -------------------------------
    def plot_current(self):
        if not self.data.strikes:                              # jika tidak ada data
            messagebox.showinfo("Kosong", "Tambahkan data terlebih dahulu.")
            return

        self.plotter.plot_planes_and_poles(self.data.strikes, self.data.dips)  # plot
        self.canvas.draw()                                      # refresh canvas

    # -------------------------------
    def clear_data(self):
        self.data.clear()                                       # hapus semua
        self.listbox.delete(0, tk.END)                          # menghilangkan listbox
        self.plotter.clear()                                    # hapus plot
        self.canvas.draw()

    # -------------------------------
    def load_csv(self):
        path = filedialog.askopenfilename(filetypes=[("CSV files", "*.csv")])  # pilih CSV
        if not path:
            return

        try:
            self.data.load_csv(path)                            # membaca data CSV
            self.listbox.delete(0, tk.END)

            for s, d in zip(self.data.strikes, self.data.dips): # menampilkan data
                self.listbox.insert(tk.END, f"{s:.1f}, {d:.1f}")
            messagebox.showinfo("Success", "CSV Loaded successfully.")
        except Exception as e:
            messagebox.showerror("Error", str(e))

    # -------------------------------
    def save_plot(self):
        path = filedialog.asksaveasfilename(defaultextension=".png")  # memiliki lokasi penyimpanan
        if not path:
            return

        self.figure.savefig(path, dpi=300)                     # menyimpan gambar
        messagebox.showinfo("Saved", f"Plot saved to {path}")


# MAIN PROGRAM

if __name__ == "__main__":
    root = tk.Tk()
    app = StereonetApp(root)                                   # menjalankan aplikasi
    root.geometry("1100x700")                                  # ukuran window
    root.mainloop()                                            # loop GUI
